# 02.30.21 - RF MFCC20 (4-Fold, minority upsample)


## Prerequisites

- The dataset exists (if not, execute 02.00.01)
- The dataset features have been populated (if not, execute 02.00.02)


## Goals

In this experiment, we intend to assess the generalization capability of a RandomForest classifier using MFCC coefficients as features, through a 4-folds cross validation over the hive axis.

In reality, due to the fact that some hives in the reference dataset only present one label value (either queen of noqueen) some folds are an agregate of 2 distincts hives, but in any case, for each fold, the classifier is tested over samples belonging to hive(s) it was never trained on. 

Distribution details are provided below:

<table border="1" class="dataframe" align="left">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>fold</th>
      <th>hive</th>
      <th>queen</th>
      <th>count(*)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
      <td>CF001</td>
      <td>0.0</td>
      <td>14</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1</td>
      <td>CF003</td>
      <td>1.0</td>
      <td>3649</td>
    </tr>
    <tr>
      <th>2</th>
      <td>2</td>
      <td>CJ001</td>
      <td>0.0</td>
      <td>790</td>
    </tr>
    <tr>
      <th>3</th>
      <td>2</td>
      <td>GH001</td>
      <td>1.0</td>
      <td>1396</td>
    </tr>
    <tr>
      <th>4</th>
      <td>3</td>
      <td>Hive1</td>
      <td>0.0</td>
      <td>1473</td>
    </tr>
    <tr>
      <th>5</th>
      <td>3</td>
      <td>Hive1</td>
      <td>1.0</td>
      <td>2684</td>
    </tr>
    <tr>
      <th>6</th>
      <td>4</td>
      <td>Hive3</td>
      <td>0.0</td>
      <td>6545</td>
    </tr>
    <tr>
      <th>7</th>
      <td>4</td>
      <td>Hive3</td>
      <td>1.0</td>
      <td>654</td>
    </tr>
  </tbody>
</table>
<br><br><br><br><br><br><br><br><br><br><br><br><br>


For some folds (see table below), queen/noqueen representation is heavily unbalanced

<table border="1" class="dataframe" align="left">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>fold</th>
      <th>Q</th>
      <th>NQ</th>
      <th>Q_ratio</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
      <td>3649</td>
      <td>14</td>
      <td>99.62%</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2</td>
      <td>1396</td>
      <td>790</td>
      <td>63.86%</td>
    </tr>
    <tr>
      <th>2</th>
      <td>3</td>
      <td>2684</td>
      <td>1473</td>
      <td>64.57%</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4</td>
      <td>654</td>
      <td>6545</td>
      <td>9.08%</td>
    </tr>
  </tbody>
</table>
<br><br><br><br><br><br><br><br>

**To mitigate this fact, for each fold we randomly upsample the under-represented class in order to reach the same cardinality as the majority class.**


## Conclusion

**The RF classifier demonstrates extremely poor generalization, regardless of the parameters we have tested so far.**


<hr style="border:1px solid gray"></hr>

### Step 1: Get previously created dataset

In [1]:
import warnings                      # This block prevents display of harmless warnings, but should be
warnings.filterwarnings('ignore')    # commented out till the experiment final version,
                                     # in order to avoid missing "real" warnings 

import kilroy_was_here               # Mandatory. Allow access to shared python code from repository root
from audace.jupytools import (
    iprint,                          # timestamped (to the ms) print with CPU and RAM consumption information  
    predestination,                  # Seeds various PRNGs for reproducibility
    say_my_name                      # gets notebook name
)

from audace.audiodataset import AudioDataset      # Main class for audio dataset handling

from IPython.display import display

#########################
# Experiment parameters #
#########################

EXP_NAME = say_my_name()  # Experiment name will be used to create outputs directory

DATASET_NAME = 'MAIN1000' # Dataset name is the master key for dataset addressing
                          # Change it according to the dataset you want to process


FEATURE_NAME = 'mfcc20'   # Name of the feature used for classification
LABEL_NAME = 'queen'      # Name of the label used for classification
FOLD_NAME = 'fold'        # Column name of the fold axis

# Initialize Dataset Object. 
ds = AudioDataset(DATASET_NAME)
    
# Display AudioDataset summary    
ds.info()

# Build dataframe containing all the information needed to conduct the experiment
sql = F"SELECT {FEATURE_NAME}, {LABEL_NAME}, {FOLD_NAME} FROM samples WHERE nobee = 0"
df = ds.queryDataFrame(sql)

# Display cardinalities by hive attribute and queen label for samples with no external perturbation
sql = """
    select distinct fold, queen, count(*)
    from samples
    where nobee = 0
    group by fold, queen
    order by fold, queen
    """
display(ds.queryDataFrame(sql))

# display distribution per fold
print("\n======  PER FOLD QUEEN/NOQUEEN DISTRIBUTION ======")
sql = """
    select distinct fold,
    count(case queen when 1.0 then 1 else null end) as Q,
    count(case queen when 0.0 then 1 else null end) as NQ,
    round(100.0*count(case queen when 1.0 then 1 else null end)/count(*), 2)||'%'  as Q_ratio
    from samples
    where nobee = 0
    group by fold
    order by fold
    """
display(ds.queryDataFrame(sql))


# display global distribution
print("\n======  GLOBAL QUEEN/NOQUEEN DISTRIBUTION ======")
sql = """
    select
    count(case queen when 1.0 then 1 else null end) as Q,
    count(case queen when 0.0 then 1 else null end) as NQ,
    round(100.0*count(case queen when 1.0 then 1 else null end)/count(*), 2)||'%'  as Q_ratio
    from samples
    where nobee = 0
    """
display(ds.queryDataFrame(sql))

[2020-09-08/13:58:12.988|14.6%|86.1%|0.28GB] ------------------------------------------------------
[2020-09-08/13:58:12.988|00.0%|86.1%|0.28GB] DATASET NAME          : MAIN1000
[2020-09-08/13:58:12.988|00.0%|86.1%|0.28GB] DATASET PATH          : D:\Jupyter\ShowBees\datasets\MAIN1000
[2020-09-08/13:58:12.988|00.0%|86.1%|0.28GB] DATASET DB PATH       : D:\Jupyter\ShowBees\datasets\MAIN1000\MAIN1000.db
[2020-09-08/13:58:12.989|00.0%|86.1%|0.28GB] DATASET SAMPLES PATH  : D:\Jupyter\ShowBees\datasets\MAIN1000\samples
[2020-09-08/13:58:12.989|00.0%|86.1%|0.28GB] NB SOURCE AUDIO FILES : 48
[2020-09-08/13:58:12.989|00.0%|86.1%|0.28GB] SAMPLE RATE           : 22050
[2020-09-08/13:58:12.989|00.0%|86.1%|0.28GB] DURATION              : 1.0
[2020-09-08/13:58:12.989|00.0%|86.1%|0.28GB] OVERLAP               : 0.0
[2020-09-08/13:58:12.989|00.0%|86.1%|0.28GB] NB AUDIO CHUNKS       : 24788
[2020-09-08/13:58:12.989|00.0%|86.1%|0.28GB] ------------------------------------------------------


,fold,queen,count(*)
0,1,0.0,14
1,1,1.0,3649
2,2,0.0,790
3,2,1.0,1396
4,3,0.0,1473
5,3,1.0,2684
6,4,0.0,6545
7,4,1.0,654



======  PER FOLD QUEEN/NOQUEEN DISTRIBUTION ======


,fold,Q,NQ,Q_ratio
0,1,3649,14,99.62%
1,2,1396,790,63.86%
2,3,2684,1473,64.57%
3,4,654,6545,9.08%



======  GLOBAL QUEEN/NOQUEEN DISTRIBUTION ======


,Q,NQ,Q_ratio
0,8383,8822,48.72%


<hr style="border:1px solid gray"></hr>

### Step 2: Process RF learning and display performance indicators

In [2]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from audace.splitters import splitTrainTestFold

#########################
# Experiment parameters #
#########################


# Seed the various PRNGs
predestination()


X_tests = []
y_tests = []
clfs = []

# Iterate over folds
for fold in ds.listAttributeValues(FOLD_NAME):
    print(F"############### FOLD {fold} ###############")
    # Build training and test datasets
    iprint(">>>>> Building partitions training/test")
    X_train, X_test, y_train, y_test = splitTrainTestFold(
        df,
        FEATURE_NAME,
        LABEL_NAME,
        FOLD_NAME,
        fold,
        balance_strategy=1 # upsample the under represented class
    )

    # Standardize data 
    iprint(">>>>> Standardize")
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    #Create a SVM Classifier, using the experiment parameters
    clf = RandomForestClassifier()

    #Train the model using the training sets
    iprint('>>>>> Train')
    clf.fit(X_train, y_train)
    
    #Store data (will be used later for reporting)
    X_tests.append(X_test)
    y_tests.append(y_test)
    clfs.append(clf)    
    

    #Predict the response for test dataset
    iprint('>>>>> Predict')
    y_pred = clf.predict(X_test)

    # Display information about the classifier performance
    iprint(F"Trained over {len(y_train)} / Tested over {len(y_test)}")
    iprint("Accuracy  :",metrics.accuracy_score(y_test, y_pred))
    iprint("Precision :",metrics.precision_score(y_test, y_pred))
    iprint("Recall    :",metrics.recall_score(y_test, y_pred))
    iprint("F-Measure :",metrics.f1_score(y_test, y_pred))


############### FOLD 1 ###############
[2020-09-08/13:58:13.775|15.0%|88.0%|0.35GB] >>>>> Building partitions training/test
[2020-09-08/13:58:13.913|19.4%|89.1%|0.44GB] >>>>> Standardize
[2020-09-08/13:58:14.308|22.9%|88.1%|0.44GB] >>>>> Train
[2020-09-08/13:59:09.788|18.8%|88.3%|0.44GB] >>>>> Predict
[2020-09-08/13:59:09.938|33.3%|88.3%|0.44GB] Trained over 17616 / Tested over 7298
[2020-09-08/13:59:09.940|00.0%|88.3%|0.44GB] Accuracy  : 0.46697725404220336
[2020-09-08/13:59:09.945|00.0%|88.3%|0.44GB] Precision : 0.0
[2020-09-08/13:59:09.949|40.0%|88.3%|0.44GB] Recall    : 0.0
[2020-09-08/13:59:09.953|00.0%|88.3%|0.44GB] F-Measure : 0.0
############### FOLD 2 ###############
[2020-09-08/13:59:09.954|00.0%|88.3%|0.44GB] >>>>> Building partitions training/test
[2020-09-08/13:59:10.091|23.6%|88.5%|0.45GB] >>>>> Standardize
[2020-09-08/13:59:10.543|39.3%|88.6%|0.45GB] >>>>> Train
[2020-09-08/13:59:53.854|21.5%|88.2%|0.45GB] >>>>> Predict
[2020-09-08/13:59:53.922|35.0%|88.2%|0.45GB] Traine

<hr style="border:1px solid gray"></hr>

### Step 3: Display performance report

TODO